# STUDIO PRELIMINARE

#### Maraziti Matteo, Tocci Federico e Scordino Giacomo

```
Le istruzione presentate di seguito sono state utilizzate per la realizzazione dei primi test.  L'obiettivo è stato quello di verificare il corretto funzionamento dell'algoritmo e di individuare i moduli di interesse.
Con l'utilizzo di ViewerGL è stato possibile visualizzare graficamente il risultato delle avvenute elaborazioni.
```

In [24]:
using ViewerGL
GL = ViewerGL
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using SparseArrays

include("../src/arrangement/planar_arrangement.jl") 

V = [2 2; 1.5 2; 3 3.5; 1 3; 5 3; 1 2; 5 2]
EV = SparseArrays.sparse(Array{Int8, 2}([
               [1 1 0 0 0 0 0] #1->1,2
               [0 1 1 0 0 0 0] #2->2,3
               [1 0 1 0 0 0 0] #3->1,3
               [0 0 0 1 1 0 0] #4->4,5
               [0 0 0 0 0 1 1] #5->6,7
           ]))
sigma=spzeros(Int8, 0)
return_edge_map=false
multiproc=false
W = convert(Lar.Points,V')

V, copEV, copFE=planar_arrangement(V,EV,sigma,return_edge_map,multiproc)

EV = Lar.cop2lar(copEV)
FE = Lar.cop2lar(copFE)
FV=EV
FV = convert(Lar.Cells,FV)
VV = [[k] for k=1:size(V,2)]
model = (W, [VV,EV,FV])::Lar.LARmodel;
meshes = GL.numbering(1.5)(model, GL.COLORS[1], 0.1);
GL.VIEW(meshes);

[RisultatoTest](/docs/src/RisultatoTest.PNG)

### Analisi Tempistiche

Dopo aver constatato il corretto funzionamento del codice principale bisogna effettuare analisi in termini di tempistiche di esecuzione dell'algoritmo.

La funzione planar_arrangement è costituita da un codice molto minimale, in quanto la parte più importante dell'esecuzione avviene nelle funzioni esterne che sono chiamate all'interno di essa. 

In [22]:
function planar_arrangement(V::Lar.Points, copEV::Lar.ChainOp, sigma::Lar.Chain=spzeros(Int8, 0),
    return_edge_map::Bool=false,
    multiproc::Bool=false)
#planar_arrangement_1
V,copEV,sigma,edge_map=planar_arrangement_1(V,copEV,sigma,return_edge_map,multiproc)
# cleandecomposition
if sigma.n > 0
    V,copEV=cleandecomposition(V, copEV, sigma, edge_map)
end
bicon_comps = Lar.Arrangement.biconnected_components(copEV)
if isempty(bicon_comps)
    println("No biconnected components found.")
    if (return_edge_map)
        return (nothing, nothing, nothing, nothing)
    else
        return (nothing, nothing, nothing)
    end
end
#Planar_arrangement_2
V,copEV,FE=Lar.Arrangement.planar_arrangement_2(V,copEV,bicon_comps,edge_map,sigma)
if (return_edge_map)
     return V, copEV, FE, edge_map
else
     return V, copEV, FE
end
end

planar_arrangement (generic function with 4 methods)

Le funzioni chiamate internamente alla funzione planar_arrangement sono:
1. planar_arrangement_1
2. cleandecomposition
3. biconnected_components
4. planar_arrangement_2

È possibile quindi pensare come sia possibile migliorare le tempistiche andando ad operare internamente alle presenti funzioni.

### Conclusioni

All'interno di ciascuna di queste funzioni vi sono ulteriori chiamate a funzioni esterne, in queste ultime si identifica come fattor comune che innalza notevolmente i tempi di esecuzione la vasta presenza di cicli for, i quali dovranno essere eseguiti per ciascun elemento da elaborare. 
Come ovviare questo problema verrà discusso all'interno del [Notebook finale](NotebookFinale.ipynb).